In [22]:
# check data
import pandas as pd
df1 = pd.read_csv("Rewriting_Questions_Statements.csv", header =0)
print(len(df1.iloc[:,0]))

11


## Increase the dataset with paraphrasing model

In [1]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MarianMTModel, MarianTokenizer
from transformers import BartForConditionalGeneration, BartTokenizer

In [2]:
# source from: https://github.com/Vamsi995/Paraphrase-Generator
# This model is trained on the Google's PAWS Dataset.
class ParaphraseMachine:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")  
        self.model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self.model.to(self.device)
        

    def paraphrase(self, input_sentence, num_of_sentences = 1):
        #encoding = self.tokenizer.encode_plus(input_sentence, padding=True, return_tensors="pt")
        encoding = self.tokenizer(input_sentence, padding=True, return_tensors="pt")
        
        input_ids, attention_masks = encoding["input_ids"].to(self.device), encoding["attention_mask"].to(self.device)


        outputs = self.model.generate(
            input_ids=input_ids, attention_mask=attention_masks,
            max_length=256,
            do_sample=True,
            top_k=120,
            top_p=0.95,
            early_stopping=True,
            repetition_penalty=0.8, # adjust this as we see fit to avoid repetition in sentences
            num_return_sequences=num_of_sentences
        )

        output_list = []
        for output in outputs:
            decoded_sentence = self.tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            output_list.append(decoded_sentence)

        return output_list

In [5]:
pm = ParaphraseMachine()
# input_sentence = df1.iloc[10,0]
input_sentence = "Sometimes it's hard to think of a creative domain that you'd like to pursue, but it's all right, don't worry! We'll figure something out for you. Here are a few exercises you can do to reconnect with your childhood and ignite a creative domain."
print(input_sentence)
#input_sentence =  "paraphrase: " + input_sentence + " </s>"

# generated_sentence = pm.batch_paraphrase(input_sentence, 5)
generated_sentence = pm.paraphrase(input_sentence, num_of_sentences=10)
print(generated_sentence)

Sometimes it's hard to think of a creative domain that you'd like to pursue, but it's all right, don't worry! We'll figure something out for you. Here are a few exercises you can do to reconnect with your childhood and ignite a creative domain.
["Sometimes it's hard to think of a creative domain that you'd like to pursue, but it's all right, don't worry we'll figure something out for you : Here are a few exercises you can do to reconnect with your childhood and ignite a creative domain's inspirations.", "Sometimes it's hard to think of a creative field that you'd like to pursue but it's all right, don't worry, we'll figure it out for you here are a few exercises you can do to reconnect with your childhood and ignite a creative domain.", "Sometimes it's hard to think of a creative domain that you would like to pursue - but it's all right - don't worry we'll figure something out for you. Here are a few exercises you can do to reconnect with your childhood and ignite a creative domain.", 